In [ ]:
#1. 사용환경 준비
import os
from dotenv import load_dotenv
from openai import OpenAI 

# openai API키 입력
load_dotenv()

client = OpenAI(api_key = os.getenv("OPENAI_API_KEY"))

In [2]:
#2. 모델 로드하기 
from langchain_openai import ChatOpenAI

# 모델 초기화
model = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
#3. 문서 로드하기
from langchain.document_loaders import CSVLoader

# PDF 파일 로드. 파일의 경로 입력
loader = CSVLoader("문화재_크롤링_결과.csv",encoding='UTF8')
loader2 = CSVLoader("여행지_크롤링_결과.csv",encoding='UTF8')

# 페이지 별 문서 로드
Treasures = loader.load()
Travel = loader2.load()

In [57]:
#4. 문서 청크로 나누기(CharacterTextSplitter)
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document



# 텍스트 청크 분할기 설정 (문단 기준 분할)
text_splitter = CharacterTextSplitter(
    chunk_size=100,
    separator="\n",
    chunk_overlap=0,
    length_function=len,
    is_separator_regex=False,
)

Treasures_splits = text_splitter.split_documents(Treasures) # 문서를 청크로 분할
Travel_splits = text_splitter.split_documents(Travel)

splits_sum = Treasures_splits + Travel_splits

Created a chunk of size 150, which is longer than the specified 100
Created a chunk of size 107, which is longer than the specified 100
Created a chunk of size 111, which is longer than the specified 100
Created a chunk of size 123, which is longer than the specified 100
Created a chunk of size 107, which is longer than the specified 100
Created a chunk of size 114, which is longer than the specified 100
Created a chunk of size 112, which is longer than the specified 100
Created a chunk of size 110, which is longer than the specified 100
Created a chunk of size 111, which is longer than the specified 100
Created a chunk of size 113, which is longer than the specified 100
Created a chunk of size 112, which is longer than the specified 100
Created a chunk of size 120, which is longer than the specified 100
Created a chunk of size 111, which is longer than the specified 100
Created a chunk of size 127, which is longer than the specified 100
Created a chunk of size 107, which is longer tha

In [58]:
#5 벡터 임베딩 생성
from langchain_openai import OpenAIEmbeddings

# OpenAI 임베딩 모델 초기화
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

In [60]:
#6. 벡터 스토어 생성
import faiss
from langchain_community.vectorstores import FAISS

# 문서에서 벡터 저장소 생성
vectorstore = FAISS.from_documents(documents=splits_sum, embedding=embeddings)


In [62]:
from langchain.vectorstores.base import VectorStore
#7. FAISS를 Retriever로 변환
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 1})

In [74]:
#8. 프롬프트 템플릿을 정의하라
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# 프롬프트 템플릿 정의
contextual_prompt = ChatPromptTemplate.from_messages([
    ("system", "문화재를 물어보면 그 문화재의 정보와 주소를 알려줘."),
    ("user", "Context: {context}\\n\\nQuestion: {question}")
])

In [75]:
#9. LangChain의 모델과 프롬프트를 연결하여 RAG 체인을 구성
from langchain.chains import LLMChain

class SimplePassThrough:
    def invoke(self, inputs, **kwargs):
        return inputs

class ContextToPrompt:
    def __init__(self, prompt_template):
        self.prompt_template = prompt_template
    
    def invoke(self, inputs):
        # 문서 내용을 텍스트로 변환
        if isinstance(inputs, list):
            context_text = "\n".join([doc.page_content for doc in inputs])
        else:
            context_text = inputs
        
        # 프롬프트 템플릿에 적용
        formatted_prompt = self.prompt_template.format_messages(
            context=context_text,
            question=inputs.get("question", "")
        )
        return formatted_prompt

# Retriever를 invoke() 메서드로 래핑하는 클래스 정의
class RetrieverWrapper:
    def __init__(self, retriever):
        self.retriever = retriever

    def invoke(self, inputs):
        if isinstance(inputs, dict):
            query = inputs.get("question", "")
        else:
            query = inputs
        # 검색 수행
        response_Treasures = self.retriever.get_relevant_documents(query)
        return response_Treasures
#텍스트 생성 체인 생성
llm_chain = LLMChain(llm=model, prompt=contextual_prompt)

# RAG 체인 설정
rag_chain_debug = {
    "context": RetrieverWrapper(retriever),
    "prompt": ContextToPrompt(contextual_prompt),
    "llm": model
}

In [73]:
# 챗봇 구동
while True:
    print("========================")
    query = input("질문을 입력하세요 : ")
    if query == "종료": # 종료 입력 시 챗봇 종료
        break
    
    # 1. Retriever로 관련 문서 검색
    response_Treasures = rag_chain_debug["context"].invoke({"question": query})
    
    # 2. 문서를 프롬프트로 변환
    prompt_messages = rag_chain_debug["prompt"].invoke({
        "context": response_Treasures,
        "question": query
    })
    
    # 3. LLM으로 응답 생성
    response = rag_chain_debug["llm"].invoke(prompt_messages)
    
    print("\n질문:",query)
    print("\n답변:")
    print(response.content) # 답변 출력


질문: 대한민국 국보 100호

답변:
대한민국 국보 100호

질문: 대한민국 국보 1호

답변:
0: 국보  
number: 201호

질문: 훈민정음

답변:
훈민정음(訓民正音)(Hunminjeongeum (The Proper Sounds for the Instruction of the People))

질문: 대한민국 국보 10호

답변:
0: 국보  
number: 10호

질문: 시발련아

답변:
죄송합니다, 요청하신 내용을 이해하지 못했습니다. 도움이 필요하시면 구체적인 질문을 해주시면 감사하겠습니다.

질문: 숭례문

답변:
숭례문

질문: 야

답변:
서울 옥천암 마애보살좌상(서울 玉泉庵磨崖菩薩坐像)

질문: 씨발

답변:
죄송하지만, 도움이 필요하신 내용을 말씀해 주시면 최선을 다해 도와드리겠습니다.
